Importing pandas ; Importing the final progress dataframe ; Importing seaborn ; Importing numpy ; Importing the average closing price ; Importing the price csv ; Setting price csv index ; Preparing the final dataframe for prediction

In [94]:
import pandas as pd
import seaborn as sns
import numpy as np

df_closing_prediction=pd.read_csv('../../data/output/price-prediction-by-book-value-growth.csv',sep=',')
df_closing_prediction=df_closing_prediction.set_index('Year')[['3 Prediction']]

df_price=pd.read_csv('../../data/input/price.csv',sep=',')
df_price['Date']=pd.to_datetime(df_price['Date'])
df_price['Year']=df_price['Date'].dt.year
df_price['Month']=df_price['Date'].dt.month
df_price=df_price.set_index(['Year','Month']).drop(columns=['Date','Adj Close','Volume'])

df_final=pd.read_csv('../../data/output/monthly-progress-by-book-value/mean/AR.csv',sep=';')[['Year','Month','AR 5 Mean']]
df_final=df_final.set_index(['Year','Month'])

df_final=pd.merge(df_price,df_final,how='left',left_index=True,right_index=True)
df_final=pd.merge(df_final,df_closing_prediction,how='left',left_index=True,right_index=True)
df_final=df_final.dropna()

df_final['Estimated Price']=df_final['AR 5 Mean']*df_final['3 Prediction']
df_final=df_final.drop(columns=['AR 5 Mean','3 Prediction'])

print(df_final.head())

              Open    High     Low   Close  Estimated Price
Year Month                                                 
2017 1      3100.0  3100.0  3100.0  3100.0      3069.772727
     1      3100.0  3155.0  3045.0  3155.0      3069.772727
     1      3170.0  3175.0  3130.0  3140.0      3069.772727
     1      3140.0  3155.0  3125.0  3135.0      3069.772727
     1      3090.0  3140.0  3090.0  3120.0      3069.772727


Prompting investor buy and sell deviation from AR mean

In [95]:
# the deviation from error mean
deviation_mean=0.029067
deviation_buy=0.01
deviation_sell=0.01

# security tax
security_tax=0.004

Generating buy price and sell price columns

In [96]:
df_final['Buy Price']=df_final['Estimated Price']*(1-deviation_buy)
df_final['Sell Price']=df_final['Estimated Price']*(1+deviation_sell)

Defining profitability apply function

In [97]:
hold_stock=False

def determine_takeProfit(row):
    
    global hold_stock
    
    if hold_stock:
        if row['Sell Price']<=row['High']:
            hold_stock=False
            return True
    else:
        if row['Buy Price']>=row['Low']:
            hold_stock=True
    
    return False
    

Applying profitability function to final dataframe

In [98]:
df_final['Take Profit']=df_final.apply(lambda row: determine_takeProfit(row), axis=1)

print(df_final['Take Profit'].value_counts())

Take Profit
False    1735
True       54
Name: count, dtype: int64


Exporting the final dataframe to csv

In [99]:
print(df_final.head())
df_final=df_final[['High','Low','Buy Price','Sell Price','Take Profit']]
df_final.to_csv('../../data/output/monthly-progress-by-book-value/profitability.csv',sep=';')

              Open    High     Low   Close  Estimated Price  Buy Price  \
Year Month                                                               
2017 1      3100.0  3100.0  3100.0  3100.0      3069.772727   3039.075   
     1      3100.0  3155.0  3045.0  3155.0      3069.772727   3039.075   
     1      3170.0  3175.0  3130.0  3140.0      3069.772727   3039.075   
     1      3140.0  3155.0  3125.0  3135.0      3069.772727   3039.075   
     1      3090.0  3140.0  3090.0  3120.0      3069.772727   3039.075   

             Sell Price  Take Profit  
Year Month                            
2017 1      3100.470455        False  
     1      3100.470455        False  
     1      3100.470455        False  
     1      3100.470455        False  
     1      3100.470455        False  
